# run lenet model
- build, train and test the model
- save the checkpoint
- add some tensor to collection

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial,name='weights')

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial,name='biases')
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [5]:
tf.reset_default_graph()
def model(x):
    # layer 1
    with tf.variable_scope('conv1'):
        W_conv1 = weight_variable([5, 5, 1, 32])
        b_conv1 = bias_variable([32])
        x_image = tf.reshape(x, [-1,28,28,1])
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)

    # layer 2
    with tf.variable_scope('conv2'):
        W_conv2 = weight_variable([5, 5, 32, 64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
    
    with tf.variable_scope('fc1'):
        W_fc1 = weight_variable([7 * 7 * 64, 1024])
        b_fc1 = bias_variable([1024])
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        
    h_fc1_drop = h_fc1
    with tf.variable_scope('fc2'):
        W_fc2 = weight_variable([1024, 10])
        b_fc2 = bias_variable([10])
        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
    return y_conv

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])        
tf.add_to_collection('x',x)
tf.add_to_collection('y',y_)

y_conv = model(x)
tf.add_to_collection('conv_out',y_conv)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
tf.add_to_collection('train_op',train_step)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.add_to_collection('loss',cross_entropy)
tf.add_to_collection('accuracy',accuracy)

In [6]:
for i in tf.trainable_variables():
    print(i.name,i.get_shape())

(u'conv1/weights:0', TensorShape([Dimension(5), Dimension(5), Dimension(1), Dimension(32)]))
(u'conv1/biases:0', TensorShape([Dimension(32)]))
(u'conv2/weights:0', TensorShape([Dimension(5), Dimension(5), Dimension(32), Dimension(64)]))
(u'conv2/biases:0', TensorShape([Dimension(64)]))
(u'fc1/weights:0', TensorShape([Dimension(3136), Dimension(1024)]))
(u'fc1/biases:0', TensorShape([Dimension(1024)]))
(u'fc2/weights:0', TensorShape([Dimension(1024), Dimension(10)]))
(u'fc2/biases:0', TensorShape([Dimension(10)]))


In [ ]:
steps=[]
train_accs = []
test_accs=[]
losses = []

# saver
saver = tf.train.Saver(max_to_keep=None)
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(500):
        batch = mnist.train.next_batch(50)
        if i<50 or i%10 == 0:
            train_accuracy,lo = sess.run([accuracy,cross_entropy],feed_dict={x:batch[0], y_: batch[1]})
            test_accuracy = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
            print("step %d--loss:%.4g, \t training accuracy %g\t test accuracy %g"%(i, lo,train_accuracy,test_accuracy))
            steps.append(i)
            train_accs.append(train_accuracy)
            test_accs.append(test_accuracy)
            losses.append(lo)

            save_path = saver.save(sess, "./mnist_lenet/model",global_step=i)
            print("Model saved in file: %s" % save_path)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

# save the model accuracy

In [6]:
import pickle
output = open('./mnist_lenet/loss_tracc_teacc.pkl', 'wb')
pickle.dump([steps,losses,train_accs,test_accs],output)
output.close()

In [7]:
infile = open('./mnist_lenet/loss_tracc_teacc.pkl','rb')
for _,l,tr,te in zip(*pickle.load(infile)):
    print l,tr,te

9.87258 0.14 0.0936
8.43879 0.12 0.1049
7.23856 0.12 0.1274
5.73586 0.12 0.1418
5.33568 0.1 0.1441
3.86052 0.1 0.1393
3.31845 0.14 0.1621
2.99157 0.26 0.1932
3.06676 0.18 0.1921
3.03526 0.16 0.1963
3.08372 0.06 0.1913
2.69055 0.2 0.1933
3.16267 0.1 0.2034
2.7132 0.24 0.2322
2.36252 0.28 0.2806
1.87746 0.34 0.3371
1.77628 0.34 0.3861
1.8036 0.46 0.4179
1.44111 0.52 0.4302
1.80516 0.38 0.4375
2.13833 0.34 0.45
1.36921 0.64 0.4609
1.65726 0.58 0.4715
1.68778 0.46 0.4766
1.48131 0.46 0.4843
1.67813 0.4 0.4979
1.3135 0.56 0.5292
1.10703 0.7 0.5654
1.48637 0.54 0.5872
1.28923 0.6 0.5949
1.17649 0.62 0.5931
0.896269 0.78 0.5892
1.03441 0.68 0.5985
1.51701 0.6 0.6092
1.15053 0.56 0.6292
1.12986 0.6 0.6471
1.53429 0.54 0.6601
1.18873 0.68 0.6701
1.06961 0.64 0.6743
0.79784 0.82 0.6847
1.13774 0.66 0.6994
0.688797 0.72 0.7117
0.846515 0.74 0.7199
1.01146 0.7 0.7257
1.11819 0.56 0.7255
1.12986 0.6 0.7183
0.950604 0.66 0.7175
1.10079 0.64 0.7219
0.944985 0.72 0.7326
1.01095 0.68 0.7389
0.803378 0.